# Introduction

Automatic Differentiation allows for fast and precise derivitative computation. It is not, however, the only method for computing derivatives. 

Derivatives are a fundamental tool in mathematics with applications in a range of scientific fields.  Differentiation is core to many computational problems including finding extrema and zeroes of functions. Optimization problems such as gradient descent in neural networks depend on partial derivatives to incrementally move towards optimal solutions. Directional derivatives and the Jacobian matrix are utilized in a variety of contexts including backpropogation in training neural networks. Directional derivatives compute the change in a mult-dimensional function with a respect to any linear combination of its input parameters. 

Because of the prevalence of derivatives in computational fields, automatic differentiation focuses on developing algorithmic techniques for efficiently evaluating derivatives.


### Symbolic differentiation
Symbolic differentiation applies basic derivative rules to compute derivatives at the expression level. While precise, symbolic differentiation uses excess computational power and memory to hold the swelling equation resulting from chain rule. Symbolic differentiation computes the derivative of a function analytically, which offers high precision at the cost of execution time and exponential size in relation to the original function. Since many science applications care about the derivative evaluated at a value, we can often bypass the need for an analytic form for the derivative.

### Finite difference method
The finite difference method, on the other hand, estimates the derivative by computing the slope of the secant line, using a small change in $x$. The precision of the derivative depends on the choice of $x$; too large of a step leads to precision error but too small of a step creates rounding error.  Numerical differentiation, on the other hand, offers O(n) time computation for an n-dimensional gradient (think multi-dimensional derivative). It achieves this by estimating the derivative instead of computing it precisely. The trade-off, as with all estimations, is a reduction in accuracy due to rounding and truncation.

### Why AutoDiff?
Automatic Differentiation achieves the precision of symbolic differentiation with time complexity that matches the complexity of computing the original function. It does this by leveraging a useful fact: when computing a derivative at a specific point, we don't need the symbolic expression of the derivative. This allows us to compute our derivative via a series of computations that compute each depending on the prior computation rather than deriving the entire expression and plugging in the point at the end.

In forward mode, the graph can be abstract as computations are made on-the-fly and no storage of all prior computations is needed. In reverse-mode, on the other hand, space needs will exceed those of computing the original function as the computational graphs of derivatives will need to be stored and computed in reverse.




## Background

Atomatic differentiation (AD) is based on the property of decomposition of a function into intermediate steps based on the chain rule: $$\frac{dy}{dx} = \frac{dy}{du} \cdot \frac{du}{dx}$$
Consider the function $f(x_1, x_2) = e^{-(\sin(x_1) + \cos(x_2)}$. 

![](https://drive.google.com/uc?export=view&id=1q8eWiv6gv-ty2rv-S0OuEOqwmeVhsLgq)

The derivative is derived by chaining together the product of the outer derivative with each of the inner derivatives. This creates a computational graph, where each edge represents an elementary operation applied to an intermediary node.

Forward mode AD uses forward accumulation of these intermediary steps, traversing the chain rule in a partial ordering from the most inner part (the original inputs) working outward until the entire derivative has been evaluated. In addition, at each step we compute the derivative of the intermediate variables, called the tangent trace. This allows for us to compute the directional derivative in a single pass.

Dual numbers are convenient for differentiation since can allow us to capture both the primal and tangent trace at the same time. A dual number can be expressed as $a + b\epsilon$, where $a$ and $b$ are real numbers and $\epsilon$ is small enough such that $\epsilon^2 = 0$. The real part stores the primal trace while the dual part stores the tangent trace.

Forward mode automatic differentiation takes a seed vector which determines the direction of the derivative. This can be any linear combination of derivatives. To compute the full Jacobian using Forward Mode AD, this requires performing $m$ passes where $m$ is the number of inputs to the original function. On each pass, we calculate the derivative with respect to that direction.

In contrast, Reverse mode AD enables us to compute the full gradient in two passes. This requires storing an explicit computational graph. In the forward pass, reverse mode computes the primal trace and partial derivatives with respect to the parent node. In the reverse pass, these are reconstructed via the chain rule by accumulating the values of all child nodes.

Runtime of Forward and Reverse modes -

When a function is defined as $ \mathbb{R}^n \rightarrow \mathbb{R}^m $

Forward Mode computes the gradient with respect to the number of independent variables.
Therefore, the computational cost of using Forward Mode depends on the number of independent variables n.

Reverse Mode computes the gradient with respect to the number of outputs. Therefore, the computational cost of using Forward Mode depends on the number of outputs, m.

As a result, if there are more outputs than inputs, Forward mode is more efficient.
On the other hand, if there are more inputs than outputs, Reverse mode is more efficient.

## How to Use `AutoDiffPy`

### Installation
The package will be available through PyPI. To install, users will use 
```
pip install AutoDiffPy
```
This will also install NumPy and Pytest since they are dependencies of our package.

### Usage
Our API exposes the following functionalities
* Compute the derivative using the `find_scalar` function on a function, input vector, and seed vector
* Find the Jacobian by calling `get_jacobian` on a function and input vector

```
# simple useage of the find_scalar function
import AutoDiffPy as adp
func = lambda x1, x2: adp.Forward.sin(x1) - adp.Forward.cos(x1*x2)
input_vector = [0.56, 0.24]
seed_vector = [1, 0]
print(adp.Forward.find_scalar(func, input_vector, seed_vector))
```

```
# simple useage of the get_jacobian function
import AutoDiffPy as adp
func = lambda x1, x2: [adp.Forward.sin(x1) - adp.Forward.cos(x1*x2), x1*x2, (x1**2)+(x2**2)]
input_vector = [0.56, 0.24]
print(adp.Forward.get_jacobian(func, input_vector))
```

## Software Organization 

Our initial software organization plan is to store our AutoDiff classes in a source folder, our Test Suite in a test folder, an example script in an example folder, and a setup script for installing relevant packages at the root level. This provides a clear distinction between core functionality, testing, examples, and setup. 

```
team12/
├── /docs
│   └── milestone1
├── LICENSE
├── README.md
└── /src
│   └── /core
│   │   ├── base_derivatives.py (holds common basic derivatives e.g. trig)
│   │   ├── dual_number.py (holds override Dunder methods)
│   │   └── override_funcs.py (holds basic math functions for dual numbers)
│   ├── forward.py (holds forward pass class)
│   ├── reverse.py (holds reverse pass class)
│   └── /structures
│       └── graph.py (Reverse: node class)
├── /tests
│     ├── individual test files per module
│     └── test.py (runs all tests and delivers relevant output)
├── /examples
│     └── app.py
└── setup.py (runs relevant instillations)

```
At the root level of our source folder we have the base classes that form our AutoDiff implementation. Core will hold a class that includes functions that compute primitive derivatives and a dual number class with its dunder methods overriden to work as dual numbers. 

In adition to core, the source folder holds two AD method classes, forward and reverse. For reverse, we need to store the computational graph and nodes and therefore these are stored in a structures folder at the same level as reverse.

### Distribution

We will distribute AutoDiffPy via PyPI with PEP517/518.


## Implementation


### Classes

#### Base Derivatives

* Motivation: the Base Derivatives class will hold direct computations of primitive derivatives such as logarithmic, exponential, trig, and logistic functions. This will allow us to directly compute derivatives at each stage of the pass. 
* Arguments: each method will take in a function $f$ and output $f'$
* Methods: The methods will include log, exp, trig functions and there inverses, logistic.

#### Dual Number

* Motivation: the Dual Number class will construct the Dual Number objects that will be used in our Forward-Mode.
* Arguments: The `__init__` method will take in an int / float as a first argument for the real part of the Dual Number. The second argument, Dual, will be optional and default to $1$. 
* Methods: This class will have methods that override the Dunder methods of `__add__`, `__pow__`, `__mul__`, `__radd__`, `__rmul__`, `__truediv__`, `__repr__`. This operator overriding will ensure that basic mathematical operations work on Dual Numbers as well as combining them with ints and floats. Critical for the Dunder methods is that they return new Dual Numbers such that they don't return one of the inputs.

#### Override Functions

* Motivation: we need to be able to perform basic mathematical computations such as trig functions, exponentials, logarithmic, and logistic using Dual Numbers.
* Arguments: Each function will take as an argument the expression (can be a Dual Number or real number) and output the value after passing the argument through the function.
* Methods: This file will include independent methods such as: `sin(arg)`, `cos(arg)`, `exp(arg)`, `log_base(arg,base)`. These functions will operate on both Dual Numbers, capturing the Dual component as needed, and real numbers.

#### Forward

* Motivation: the Forward-Mode class runs Forward-Mode AD on a function $f$.
* Arguments: the class takes a function `f` in its `__init__` method.
* Methods:
  * compute_derivative accepts an input variable and evaluates the derivative of $f$ at the variable.
  * compute_jacobian computes the Jacobian via computing the derivative on all the input variables.   

#### Reverse (more will be expanded upon in M2: extension)

* Motivation: the Reverse-Mode class runs Reverse-Mode AD on a function $f$. 
* Arguments: the Reverse-Mode class when initialized will accept a function $f$ and a computational graph comprising Nodes.
* Methods: the Reverse-Mode class will have $sort$, get_derivative, get_jacobian, and backward_pass methods. More on these methods in M2: extension.

#### Node (more will be expanded upon in M2: extension)

* Motivation: the Node module will hold the Node class that forms the computational graph needed for Reverse-Mode AD. 
* Node:
  * Arguments: the Node will be initialized with two arguments: a value and a local gradient. It will also hold an array of child nodes to form the graph.
  * Methods: the Node class' Dunder methods will be overriden such that they return new Nodes.

### Data Structures

The data structures used will be the computational graph via the Nodes stored in the Graph class. 

### External Libraries
* Numpy will be used for computation using vectors and matrices.
* Pytest will be used for testing our classes.




## Feedback

The feedback received from Milestone 1 was positive overall and our design does not require significant updates. In this revision, we included an additional section in `Implementation` to include the override functions for sin, cos, pow, exp, etc. as suggested in the feedback. The `Software Organization` directory was also updated to reflect this additional file.

## License

Our software will be licensed under the MIT license, which is a highly permissive license and places few restrictions on reuse. As our software project uses standard methods of AD, we do not plan to pursue a patent. Since there are different auto differentiation packages available (such as in NumPy, PyTorch, TensorFlow, MatLab/Fortran (with packages called openAD, ADiMat), C++ (ADLOC), ADiMat, etc.), and we want other software engineers to use ours, we did not want to pick a license that would deter users. We did not add an explicit patent grant because we are not doing anything that has not been done before, so patents are not a consideration for us. Also note that since `numpy` and `matplotlib` have BSD licenses, we don't need to copy over their licenses.